In [6]:
from words import get_text, words, filenames
import os

# Search Engine Comparision: 

# Linear Search v.s. Hashtable Search

Let us imagine we have a collection of about 4500 files, each being an article filled with numerous words. We aim to search for specific words across all files and return a list of filenames where the file content includes all the words in terms.

We will try to carry out this objective with both a traditional linear search, as well as a method using hash tables. 

In [17]:

def filelist(root):
    root = os.path.expanduser(root)  # Expand the user directory if present
    retlist = []
    
    for root_dir, subdirs, files in os.walk(root):
        for filename in files:
            file_path = os.path.join(root_dir, filename)
            retlist.append(file_path)
    
    return retlist
files = filelist("~/data/slate/")

The *files* variable is a list which contains around ~4500 articles from the online magazine Slate in a .txt format

In [54]:
len(files)

4530

# Linear Search

terms = ['picture','perfect','for','and','because','since']


In [55]:
def linear_search(files, terms):
    retlist = []
    corpus = [words(get_text(file)) for file in files]
    for i in range(len(files)):
        if set(terms) <= set(corpus[i]):
            retlist.append(files[i])
    return filenames(retlist)


terms = ['picture','perfect','for','and','because','since']

# Time Spent: 1.3 second

In [56]:
%time linear_search(files,terms)

CPU times: user 1.19 s, sys: 113 ms, total: 1.3 s
Wall time: 1.3 s


['Article247_3653.txt',
 'Article247_4014.txt',
 'ArticleIP_1268.txt',
 'ArticleIP_1553.txt',
 'ArticleIP_2725.txt',
 'ArticleIP_2911.txt',
 'ArticleIP_2922.txt',
 'ArticleIP_2941.txt',
 'ArticleIP_4062.txt',
 'ArticleIP_20511.txt',
 'ArticleIP_25134.txt',
 'ArticleIP_25161.txt',
 'ArticleIP_32882.txt',
 'ArticleIP_38823.txt',
 'ArticleIP_56271.txt']

## Pre-define the number of buckets in your hash table:
- A limitation of hash tables is having to re-index everytime a bucket is added

In [ ]:
def htable(nbuckets):
    return [[]]*nbuckets

## Create some sort of hash function to index a bucket
- This should always return an integer
- We can then use this integer to represent the location of the value associated with that bucket

In [ ]:
def hashcode(o):
    if isinstance(o, int):
        return o
    elif isinstance(o, str):
        h = 0
        for c in o:
            h = h * 31 + ord(c)
        return h
    else:
        return None


# Reading and Writing to the Hash Table

The htable put and htable get functions allow us to read and write values from our hash table. They work by running the key through a hash function to find the key's corresponding bucket. They then check the relevant bucket to see if any of the key, value pair tuples have the relevant key.

If the relevant key is in the bucket, the htable_get function will return the relevant value. Meanwhile, the htable_put function will replace the value with the new value

If the relevant key isn't in the bucket, the htable_get function will return None, meanwhile the htable_put function will append a new key value pair

In [44]:

def bucket_indexof(table, key):
    return hashcode(key) % len(table)
    
def htable_put(table, key, value):
    index = bucket_indexof(table, key)
    bucket = table[index]

    # Filter out existing (key, value) pairs with the same key
    bucket = [(k, v) for k, v in bucket if k != key]

    # Append the new (key, value) pair
    bucket.append((key, value))

    # Update the bucket in the table
    table[index] = bucket

    return table

def htable_get(table, key):
    index = bucket_indexof(table, key)
    bucket = table[index]

    for pair in bucket:
        if key == pair[0]:
            return pair[1]
    return None


# Using Hash Tables as Indexes
Instead of the linear search, where you have to manually go through every file to check if the relevant search terms are present, using hashtables we can create an index. We create an index of each word to all the files they're in, with the key being the word and the value being a list of files the word is present in. 

Doing this, all we need to do to search for terms is to reference our index hash table and find the intersection of files for the relevant word indexes.

The tradeoff between the linear search and using hash tables is in the initial creation of the index. After we create the index, searching becomes much faster than linear search. But the index takes a bit to create. So which one you want to use will come down to how many times you plan on using the search operation

In [45]:
def myhtable_create_index(files):
    wordtab = htable(4011)
    for doc in files:
        wordlist = set(words(get_text(doc)))
        for word in wordlist:
            oldval = htable_get(wordtab,word)
            if oldval:
                newval = oldval
            else:
                newval = []
            newval.append(doc)
            wordtab = htable_put(wordtab,word,newval)
    return wordtab


def myhtable_index_search(files, index, terms):
    sets_list = []
    for w in terms:
        info = htable_get(index, w)
        if info:
            sets_list.append(set(info))
    
    if not sets_list:
        return sets_list

    # Calculate the intersection of all sets in the 'sets_list' using set.intersection. The * unpacks the sets
    allmatches = set.intersection(*sets_list)
    
    return allmatches.intersection(set(files))

# Time Spent: 15 seconds

In [49]:
%time index = myhtable_create_index(files)

CPU times: user 14.7 s, sys: 57.5 ms, total: 14.8 s
Wall time: 14.8 s


# Time Spent: 613 µs

1 second = 1,000,000 microseconds

In [53]:
%time myhtable_index_search(files, index, terms)

CPU times: user 609 µs, sys: 0 ns, total: 609 µs
Wall time: 613 µs


{'/home/karthik/data/slate/13/Article247_3653.txt',
 '/home/karthik/data/slate/17/Article247_4014.txt',
 '/home/karthik/data/slate/24/ArticleIP_1268.txt',
 '/home/karthik/data/slate/27/ArticleIP_1553.txt',
 '/home/karthik/data/slate/39/ArticleIP_2725.txt',
 '/home/karthik/data/slate/40/ArticleIP_2911.txt',
 '/home/karthik/data/slate/40/ArticleIP_2922.txt',
 '/home/karthik/data/slate/40/ArticleIP_2941.txt',
 '/home/karthik/data/slate/48/ArticleIP_4062.txt',
 '/home/karthik/data/slate/50/ArticleIP_20511.txt',
 '/home/karthik/data/slate/50/ArticleIP_25134.txt',
 '/home/karthik/data/slate/50/ArticleIP_25161.txt',
 '/home/karthik/data/slate/51/ArticleIP_32882.txt',
 '/home/karthik/data/slate/51/ArticleIP_38823.txt',
 '/home/karthik/data/slate/53/ArticleIP_56271.txt'}